# Imports:

In [1]:
import numpy as np
import os

# Data Preparation:

In [2]:
root_dir = r"Data/"
file_names = os.listdir(root_dir)
datasets = {}
for file in file_names:
    with open(root_dir + file) as f:
        lines = f.readlines()
        for line in lines:
            if line.startswith("ITEMS SECTION"):
                data_start = lines.index(line) + 1
                break
            elif line.startswith("CAPACITY OF KNAPSACK:"):
                cap = int(line.split("\t")[-1])
        mat = np.zeros((len(lines) - data_start, 2), dtype=np.int32)
        for i in range(data_start, len(lines)):
            (_, mat[i - data_start, 0], mat[i - data_start, 1], _) = lines[i].split('\t')
        datasets[file] = mat, cap

# Define Functions:

In [3]:
def f(x, avg_w, values, _B, _ALPHA, _DELTA, _USE_CHEBYSHEV):
    x = np.array(x, dtype=np.float64)
    avg_w = np.array(avg_w, dtype=np.float64)
    exp_sum_w = np.dot(x, avg_w)
    sum_v = np.dot(x, values)
    sig_x_i = np.sum(x, dtype=np.float64)
    delta = _DELTA
    if _USE_CHEBYSHEV: # uniform Chebyshev:
        PWB = (delta ** 2.0) * sig_x_i / ((delta ** 2.0) * sig_x_i + 3.0 * (_B - exp_sum_w) ** 2.0)
    else: # chernoff:
        if sig_x_i != 0.0:
            denum_base = ((delta * sig_x_i + _B - exp_sum_w) / (delta * sig_x_i))
            PWB = (np.exp((_B - exp_sum_w) / (delta * sig_x_i)) / np.float_power(denum_base, denum_base)) ** (sig_x_i / 2.0)
            if np.isnan(PWB):
                PWB = 1.0
        else:
            PWB = 0.0
    return max(0.0, exp_sum_w - _B), max(0.0, PWB - _ALPHA), sum_v

def lex_compare(f1, f2):
    if f1[0] > f2[0] or (f1[0] == f2[0] and f1[1] > f2[1]) or (f1[0] == f2[0] and f1[1] == f2[1] and f1[2] < f2[2]):
        return 2
    else:
        return 1

# Main Algorithm Loop:

In [13]:
_MAX_ITER = 100000
_alpha_delta_settings = [
    (0.0001, 25),
    (0.0001, 50),
    (0.001, 25),
    (0.001, 50),
    (0.01, 25),
    (0.01, 50),
]
for dataset_idx in range(len(file_names)):
    print('|' * 120)
    print(file_names[dataset_idx])
    dataset = np.array(datasets[file_names[dataset_idx]][0])
    _B_ORIG = datasets[file_names[dataset_idx]][1]
    avg_w = dataset[:, 1]
    v = dataset[:, 0]
    n = dataset.shape[0]
    k = 0
    _GAMMA = 100
    remains = _B_ORIG
    idxs = np.argsort(avg_w)
    for i in idxs:
        if remains - avg_w[i] < 0:
            break
        remains -= avg_w[i]
        k += 1
    _B = _B_ORIG + k * _GAMMA
    for i in range(avg_w.shape[0]):
        avg_w[i] += _GAMMA
    for _ALPHA, _DELTA in _alpha_delta_settings:
        for _CHEBYSHEV in [False, True]:
            # Main loop
            results = []
            for _ in range(30):
                x = np.random.randint(0, 2, (n,))
                f_x = f(x, avg_w, v, _B, _ALPHA, _DELTA, _CHEBYSHEV)
                # print("*" * 10)
                for _ in range(_MAX_ITER):
                    y = np.random.binomial(1, 1.0 / n, (n,))
                    while (y == np.zeros_like(y)).all():
                        y = np.random.binomial(1, 1.0 / n, (n,))
                    y_ = np.logical_xor(x, y)
                    f_y = f(y_, avg_w, v, _B, _ALPHA, _DELTA, _CHEBYSHEV)
                    if lex_compare(f_y, f_x) == 1:
                        x = y_
                        f_x = f_y
                print("(u(x),v(x),P(x)):\t", f_x)
                results.append(f_x[2])
                if f_x[0] != 0:
                    print("KNAPSACK Broken Error:", f_x)
            if _CHEBYSHEV:
                mod_str = "Chebyshev"
            else:
                mod_str = "Chernoff "
            print("_ALPHA:", _ALPHA, "\t_DELTA:", _DELTA, "\tMode:", mod_str, "\tProfit:", np.mean(results))
            print(results)
            print('     ' + '=' * 110)

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
eil101_n100_bounded-strongly-corr_01[1].ttp


<ipython-input-3-7d940d4e65ac>:13: RuntimeWarning: invalid value encountered in float_power
  PWB = (np.exp((_B - exp_sum_w) / (delta * sig_x_i)) / np.float_power(denum_base, denum_base)) ** (sig_x_i / 2.0)
<ipython-input-3-7d940d4e65ac>:13: RuntimeWarning: invalid value encountered in double_scalars
  PWB = (np.exp((_B - exp_sum_w) / (delta * sig_x_i)) / np.float_power(denum_base, denum_base)) ** (sig_x_i / 2.0)


(u(x),v(x),P(x)):	 (0.0, 0.0, 14754.0)
(u(x),v(x),P(x)):	 (0.0, 0.0, 14747.0)


KeyboardInterrupt: 